## Problem 1:
[Link to LLM Conversation for Balking Logic](https://chatgpt.com/share/68eda8d0-3510-8011-9a91-ae5c4779db28)

[Link to GitHub Repo for more Detailed Work on Food Truck Simulator](https://github.com/manpazito/ieor-174-assignments/tree/main/Assignment%202)

**Setting.** Suppose that you run a food truck on each Wednesday between 11AM and 1PM. There is no customer waiting when the food truck opens at 11AM. Customers arrive at the food truck according to a Poisson process with rate λ persons per minute. There is a single line of queue with a first-come-first-serve rule. Customers that have arrived before 1PM will eventually get served. In other words, customers are not able to join the queue if they arrive later than 1PM. The food truck resumes its operation until all customers who have arrived before 1PM get served. You are the owner of the food truck and there is only one server (yourself). The service time for each customer is independent and identically distributed with some distribution $F$ . The service time includes the time on taking orders and food preparation. The distribution $F$ has a mean of $1/μ$ and the probability distribution is to be specified. Suppose λ = 2. Simulate 100 independent days (Wednesdays) for each of the following parts.

**To do.** Simulate $n = 100$ independent days of operation from 11AM and 1PM and generate the sequence of customer waiting times. For this task, the service time distribution $F$ is given by an exponential distribution with expectation as 35 seconds. Recall that the condition $λ = 2$ persons per minute in the problem statement implies that the exponentially distributed inter-arrival times of customers have expectation also as 30 seconds.

In [1]:
import numpy as np
from FoodTruck import FoodTruck, format_time_from_minutes

### Default Parameters ###
LAMBDA = 2.0            # arrivals per minute
SERVICE_MEAN_SEC = 35    # average service time
T0 = 0              # 11 AM
T1 = 120            # 1 PM
N_DAYS = 100

a.) Suppose that you are interested in knowing that on a typical Wednesday, what is the expectation of the averaged waiting time for customers who arrive at the food truck between 11:15 AM to 11:30 AM. Compute an estimate of this quantity using simulation.

In [2]:
AM_TOTAL_WAIT_TIMES = []
PM_TOTAL_WAIT_TIMES = []

for _ in range(N_DAYS):
    sim = FoodTruck(LAMBDA, SERVICE_MEAN_SEC, T0, T1)
    daily_arrivals, daily_waiting, _, _ = sim.simulate_day()

    # 11:15 - 11:30 AM arrivals
    AM_MASK = (daily_arrivals >= 15) & (daily_arrivals < 30)
    AM_WAIT_TIMES = daily_waiting[AM_MASK]
    AM_TOTAL_WAIT_TIMES.extend(AM_WAIT_TIMES)   

    # 12:45 - 1:00 PM arrivals (for part b.)
    PM_MASK = (daily_arrivals >= 105) & (daily_arrivals < 120)
    PM_WAIT_TIMES = daily_waiting[PM_MASK]
    PM_TOTAL_WAIT_TIMES.extend(PM_WAIT_TIMES)
    
# Part (a)
AM_AVG_WAIT = np.mean(AM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(AM_AVG_WAIT)
print(f"Average wait time for customers arriving between 11:15 and 11:30 AM over {N_DAYS} days: {min} minutes and {sec} seconds")

Average wait time for customers arriving between 11:15 and 11:30 AM over 100 days: 6 minutes and 11 seconds


b.) Suppose that you are interested in knowing that on a typical Wednesday, what is the expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM. Compute an estimate of this quantity using simulation.

In [3]:
PM_AVG_WAIT = np.mean(PM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(PM_AVG_WAIT)
print(f"Average wait time for customers arriving between 12:45 and 1:00 PM over {N_DAYS} days: {min} minutes and {sec} seconds")

Average wait time for customers arriving between 12:45 and 1:00 PM over 100 days: 22 minutes and 36 seconds


c.) Compare the two quantities you computed in the previous two questions. Describe intuition that you may get from this comparison.

> Wait times from later in the day are much larger. Almost by 15-20 minutes longer. This is probably because customers arrive at a faster rate that one chef is able to serve.

d.) Compute the percentage of customers who arrive at the food truck between 11:15 AM to 11:30 AM and wait for more than 3 minutes. Compute the percentage of customers who arrive at the food truck between 12:45 PM to 1:00 PM and wait for more than 3 minutes.

In [4]:
AM_over_3_min = np.sum(np.array(AM_TOTAL_WAIT_TIMES) > 3)
PM_over_3_min = np.sum(np.array(PM_TOTAL_WAIT_TIMES) > 3)
print(f"Proportion of customers waiting more than 3 minutes between 11:15 and 11:30 AM: {AM_over_3_min / len(AM_TOTAL_WAIT_TIMES):.2%}")
print(f"Proportion of customers waiting more than 3 minutes between 12:45 and 1:00 PM: {PM_over_3_min / len(PM_TOTAL_WAIT_TIMES):.2%}")

Proportion of customers waiting more than 3 minutes between 11:15 and 11:30 AM: 70.84%
Proportion of customers waiting more than 3 minutes between 12:45 and 1:00 PM: 98.08%


e.) Suppose that the customers will immediately abandon the system and leave for other dining options, conditional on that they see more than 5 people in the system (including the one being served). Now, what is the percentage of customers who abandon the system (=food truck) upon arrival between 12:45 PM and 1:00 PM? What is the expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM and did not abandon the system?

In [5]:
NEW_PM_TOTAL_WAIT_TIMES = []
all_arrivals = 0
balked_arrivals = 0

for _ in range(N_DAYS):
    sim = FoodTruck(LAMBDA, SERVICE_MEAN_SEC, T0, T1)
    daily_arrivals, daily_waiting, _, balked_flags = sim.simulate_day(BALKING=True, BALKING_THRESHOLD=5)
    
    serviced_mask = (daily_arrivals > 105) & (daily_arrivals < 120) & (~balked_flags)
    balked_mask = (daily_arrivals > 105) & (daily_arrivals < 120) & (balked_flags)
    
    NEW_PM_TOTAL_WAIT_TIMES.extend(daily_waiting[serviced_mask])
    all_arrivals += np.sum(serviced_mask) + np.sum(balked_mask)
    balked_arrivals += np.sum(balked_mask)
    
NEW_AVG_WAIT = np.mean(NEW_PM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(NEW_AVG_WAIT)
print(f"Average wait time for customers arriving between 12:45 and 1:00 PM who did not balk over {N_DAYS} days: {min} minutes and {sec} seconds")
print(f"Proportion of customers who balked between 12:45 and 1:00 PM: {balked_arrivals / all_arrivals:.2%}")

Average wait time for customers arriving between 12:45 and 1:00 PM who did not balk over 100 days: 1 minutes and 42 seconds
Proportion of customers who balked between 12:45 and 1:00 PM: 22.89%


f.) Re-do the previous five parts with F being an exponential distribution with expectation as 30 seconds and everything else equal.

In [6]:
SERVICE_MEAN_SEC = 30    # new faster average service time

AM_TOTAL_WAIT_TIMES = []
PM_TOTAL_WAIT_TIMES = []

for _ in range(N_DAYS):
    sim = FoodTruck(LAMBDA, SERVICE_MEAN_SEC, T0, T1)
    daily_arrivals, daily_waiting, _, _ = sim.simulate_day()

    # 11:15 - 11:30 AM arrivals
    AM_MASK = (daily_arrivals >= 15) & (daily_arrivals < 30)
    AM_WAIT_TIMES = daily_waiting[AM_MASK]
    AM_TOTAL_WAIT_TIMES.extend(AM_WAIT_TIMES)   

    # 12:45 - 1:00 PM arrivals (for part b.)
    PM_MASK = (daily_arrivals >= 105) & (daily_arrivals < 120)
    PM_WAIT_TIMES = daily_waiting[PM_MASK]
    PM_TOTAL_WAIT_TIMES.extend(PM_WAIT_TIMES)
    
# Part (a)
AM_AVG_WAIT = np.mean(AM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(AM_AVG_WAIT)
print(f"Average wait time for customers arriving between 11:15 and 11:30 AM over {N_DAYS} days: {min} minutes and {sec} seconds")

PM_AVG_WAIT = np.mean(PM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(PM_AVG_WAIT)
print(f"Average wait time for customers arriving between 12:45 and 1:00 PM over {N_DAYS} days: {min} minutes and {sec} seconds\n")

AM_over_3_min = np.sum(np.array(AM_TOTAL_WAIT_TIMES) > 3)
PM_over_3_min = np.sum(np.array(PM_TOTAL_WAIT_TIMES) > 3)
print(f"Proportion of customers waiting more than 3 minutes between 11:15 and 11:30 AM: {AM_over_3_min / len(AM_TOTAL_WAIT_TIMES):.2%}")
print(f"Proportion of customers waiting more than 3 minutes between 12:45 and 1:00 PM: {PM_over_3_min / len(PM_TOTAL_WAIT_TIMES):.2%}")

NEW_PM_TOTAL_WAIT_TIMES = []
all_arrivals = 0
balked_arrivals = 0

for _ in range(N_DAYS):
    sim = FoodTruck(LAMBDA, SERVICE_MEAN_SEC, T0, T1)
    daily_arrivals, daily_waiting, _, balked_flags = sim.simulate_day(BALKING=True, BALKING_THRESHOLD=5)
    
    serviced_mask = (daily_arrivals > 105) & (daily_arrivals < 120) & (~balked_flags)
    balked_mask = (daily_arrivals > 105) & (daily_arrivals < 120) & (balked_flags)
    
    NEW_PM_TOTAL_WAIT_TIMES.extend(daily_waiting[serviced_mask])
    all_arrivals += np.sum(serviced_mask) + np.sum(balked_mask)
    balked_arrivals += np.sum(balked_mask)
    
NEW_AVG_WAIT = np.mean(NEW_PM_TOTAL_WAIT_TIMES)
min, sec = format_time_from_minutes(NEW_AVG_WAIT)
print(f"Average wait time for customers arriving between 12:45 and 1:00 PM who did not balk over {N_DAYS} days: {min} minutes and {sec} seconds")
print(f"Proportion of customers who balked between 12:45 and 1:00 PM: {balked_arrivals / all_arrivals:.2%}")

Average wait time for customers arriving between 11:15 and 11:30 AM over 100 days: 3 minutes and 44 seconds
Average wait time for customers arriving between 12:45 and 1:00 PM over 100 days: 8 minutes and 17 seconds

Proportion of customers waiting more than 3 minutes between 11:15 and 11:30 AM: 48.26%
Proportion of customers waiting more than 3 minutes between 12:45 and 1:00 PM: 72.94%
Average wait time for customers arriving between 12:45 and 1:00 PM who did not balk over 100 days: 1 minutes and 14 seconds
Proportion of customers who balked between 12:45 and 1:00 PM: 16.07%
